In [1]:
import pandas as pd
import os
import numpy as np
import glob
from sklearn import preprocessing

# Audio feature extraction without time series

## Data extraction

In [2]:
path = "/Users/jiachenyao/Desktop/Thesis/OpenSmile/single_emotions/"

# List all the .csv files in the folder
single_emo_files = glob.glob(path + "/*.csv")

single_emo_dfs = []

for filename in single_emo_files:
    df = pd.read_csv(filename, sep=";", index_col=None, header=0)
    
    df.drop(columns=["name","frameTime"], inplace=True)

    
            
    single_emo_dfs.append(df)
        
print(len(single_emo_dfs))

373


In [3]:
#Extract functional features
single_emo_functional_features_results = []

for df, file in zip(single_emo_dfs, single_emo_files):

    # Extract filename from filepath
    filename = os.path.basename(file)
    #print(filename)

    # Calculate functional features
    means = df.mean()
    
    result = pd.concat([means], axis=1)    

    # Rename columns with filename as prefix
    prefix = filename[:-4] # remove ".csv" extension
    result.columns = [f"{prefix}_mean"]


    single_emo_functional_features_results.append(result)
print(len(single_emo_functional_features_results))

373


In [4]:
# Concate results to a single dataframe
single_emo_audio_no_time_series_output = pd.concat(single_emo_functional_features_results, axis=1)
single_emo_audio_no_time_series_output = single_emo_audio_no_time_series_output.T
single_emo_audio_no_time_series_output = single_emo_audio_no_time_series_output.reset_index().rename(columns={'index': 'dataname'})
single_emo_audio_no_time_series_output.fillna(0,inplace=True)

In [5]:
single_emo_audio_no_time_series_output['F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope']

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
368    0.0
369    0.0
370    0.0
371    0.0
372    0.0
Name: F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope, Length: 373, dtype: float64

## Preprocessing

In [6]:
single_emo_audio_no_time = single_emo_audio_no_time_series_output.copy()
single_emo_audio_no_time["dataname"] = single_emo_audio_no_time["dataname"].apply(lambda x: x.split('_')[1])

In [7]:
# mix-max normalization
cols_to_normalize = single_emo_audio_no_time.columns.difference(['dataname'])
single_emo_audio_no_time[cols_to_normalize] = single_emo_audio_no_time[cols_to_normalize].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [8]:
single_emo_audio_no_time = single_emo_audio_no_time.dropna(axis=1, how='all')

In [9]:
# save preprocessed files
single_emo_audio_no_time.to_csv("/Users/jiachenyao/Desktop/Thesis/Preprocessed_data/single_emo_audio_no_time.csv", index=False)